In [1]:
%load_ext autoreload

In [1]:
model_file = "models_archive/model_ACAT22.py"
config_json = "training_config/training_config_awk_t3psi.json"

In [2]:
import awk_data
import awkward as ak
import tensorflow as tf
import loader
import argparse 
from collections import defaultdict
import os, json
import importlib.util
from time import time
import numpy as np
from plotting import * 
import plot_loss

config = json.load(open(config_json))

config['activation'] = tf.keras.activations.get(config['activation'])

# Checking hardware
print('version={}, CUDA={}, GPU={}'.format(
    tf.__version__, tf.test.is_built_with_cuda(),
    len(tf.config.list_physical_devices('GPU')) > 0))
      
gpus =  tf.config.list_physical_devices('GPU')

# if len(gpus) ==1 :
#     print("Using 1 GPU")
#     tf.config.experimental.set_memory_growth(gpus[0], enable=True)
#     strategy = tf.distribute.OneDeviceStrategy("gpu:0")
# elif len(gpus):
#     print("Using {} GPUs".format(len(gpus)))
#     for gpu in gpus:
#         tf.config.experimental.set_memory_growth(gpu, enable=True)
#     strategy = tf.distribute.MirroredStrategy()
# else:
#     strategy = tf.distribute.OneDeviceStrategy("cpu:0")
if len(gpus) >=1 :
    print("Using 1 GPU")
    #tf.config.experimental.set_memory_growth(gpus[0], enable=True)
    strategy = tf.distribute.OneDeviceStrategy("gpu:0")
else:
    strategy = tf.distribute.OneDeviceStrategy("cpu:0")


##################
# Prepare the output folder
def get_unique_run():
    previous_runs = os.listdir(config["models_path"])
    if len(previous_runs) == 0:
        run_number = 1
    else:
        run_number = max([int(s.split('run_')[1]) for s in previous_runs]) + 1
    return run_number

if not os.path.isdir(config["models_path"]):
    os.makedirs(config["models_path"])

name =  'run_{:02}'.format(get_unique_run())

outdir = config["models_path"] + "/"+ name

if os.path.isdir(outdir):
    print("Output directory exists: {}".format(outdir), file=sys.stderr)
else:
    os.makedirs(outdir)

print("Model output folder: ", outdir)

############################3
#Copying the config file and model file in the output dir:
os.system("cp {} {}".format(config_json, outdir))
os.system("cp {} {}".format(model_file, outdir))

###########################
## Loading the datasets
print(">>> Loading datasets")

2022-09-29 18:48:21.753698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-29 18:48:21.913658: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-29 18:48:21.913687: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-29 18:48:21.948144: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-29 18:48:23.101836: W tensorflow/stream_executor/platform/de

version=2.10.0, CUDA=True, GPU=False
Model output folder:  /work/dvalsecc/Clustering/models_archive/gcn_models/ACAT2021_v2/tests/run_28
>>> Loading datasets


2022-09-29 18:48:25.236766: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-29 18:48:25.236812: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-29 18:48:25.236831: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t3ui01.psi.ch): /proc/driver/nvidia/version does not exist
2022-09-29 18:48:25.237588: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**config["dataset_conf"]["training"]))
test_ds = awk_data.load_dataset(awk_data.LoaderConfig(**config["dataset_conf"]["validation"]))
# Create training and validation
ds_train = train_ds.prefetch(tf.data.AUTOTUNE)
#ds_train = train_ds

In [3]:
dataset = awk_data.load_tfdataset_and_original(awk_data.LoaderConfig(**config["dataset_conf"]["training"]))

In [4]:
it = iter(dataset)

In [8]:
X, original = next(it)

In [6]:
X

((<tf.Tensor: shape=(512, 23, 15), dtype=float64, numpy=
  array([[[ 9.69290151,  4.74036347,  1.82029835, ...,  3.12753276,
           -0.52162167,  1.64462817],
          [-0.2418632 , -0.36619773,  1.84469092, ..., -0.70409121,
            0.58781235, -0.51574295],
          [-0.        , -0.        , -0.        , ..., -0.        ,
           -0.        , -0.        ],
          ...,
          [-0.        , -0.        , -0.        , ..., -0.        ,
           -0.        , -0.        ],
          [-0.        , -0.        , -0.        , ..., -0.        ,
           -0.        , -0.        ],
          [-0.        , -0.        , -0.        , ..., -0.        ,
           -0.        , -0.        ]],
  
         [[ 1.88664597,  4.06202852,  0.86849889, ...,  1.19283742,
           -2.20210624,  4.02103641],
          [-0.29610849, -0.3746007 ,  0.93920084, ..., -0.55474397,
            0.58781235, -0.51574295],
          [-0.29703499, -0.37502724,  0.89935639, ..., -0.59769866,
        

In [27]:
original.cl_features.en_cluster[0].tolist()

[34.161643981933594,
 4.432665824890137,
 2.667815923690796,
 1.3657797574996948,
 0.8973453640937805,
 0.34763097763061523,
 0.3012291491031647,
 0.33249202370643616,
 0.30867254734039307,
 0.2582477033138275]

In [25]:
X[1][2][0,:,0]

<tf.Tensor: shape=(23,), dtype=float64, numpy=
array([34.16164398,  4.43266582,  2.66781592,  1.36577976,  0.89734536,
        0.34763098,  0.30122915,  0.33249202,  0.30867255,  0.2582477 ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ])>

In [5]:
a = next(it)a

In [9]:
a[2]

IndexError: tuple index out of range

In [10]:
len(a[0])

3

In [4]:
X, y, w = next(iter(ds_train))

In [ ]:
cfg = config["dataset_conf"]["training"]
cfg["maxevents"] = 512000
cfg["batch_size"] = 512
cfg["nworkers"] = 4
cfg["max_batches_in_memory"] = 50

train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**cfg))
it = iter(train_ds)
                                
result = %timeit -n 100 -r 10 -o next(it)
           

In [8]:
cfg = config["dataset_conf"]["training"]
cfg["maxevents"] = 512000
cfg["batch_size"] = 512
cfg["nworkers"] = 3
cfg["max_batches_in_memory"] = 50

train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**cfg))
it = iter(train_ds)
                                
result = %timeit -n 100 -r 10 -o next(it)
                                

65.8 ms ± 15.7 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [10]:
cfg = config["dataset_conf"]["training"]
cfg["maxevents"] = 512000
cfg["chunk_size"] = 512*20
cfg["batch_size"] = 512
cfg["nworkers"] = 3
cfg["max_batches_in_memory"] = 50

train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**cfg))
it = iter(train_ds)
                                
result = %timeit -n 100 -r 10 -o next(it)
                      

79.4 ms ± 18 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [6]:
cfg = config["dataset_conf"]["training"]
cfg["maxevents"] = 512000
cfg["chunk_size"] = 512*20
cfg["batch_size"] = 512
cfg["nworkers"] = 3
cfg["max_batches_in_memory"] = 300

train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**cfg))
it = iter(train_ds)
                                
result = %timeit -n 100 -r 10 -o next(it)

15.6 ms ± 7.16 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [8]:
cfg = config["dataset_conf"]["training"]
cfg["maxevents"] = 512000
cfg["chunk_size"] = 512*20
cfg["batch_size"] = 512
cfg["nworkers"] = 3
cfg["max_batches_in_memory"] = 300

train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**cfg))
it = iter(train_ds)
                                
result = %timeit -n 100 -r 10 -o next(it)

15.3 ms ± 5.93 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [8]:
cfg = config["dataset_conf"]["training"]
cfg["maxevents"] = 512000
cfg["chunk_size"] = 512*10
cfg["batch_size"] = 512
cfg["nworkers"] = 3
cfg["max_batches_in_memory"] = 300

train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**cfg))
it = iter(train_ds)
                                
result = %timeit -n 100 -r 10 -o next(it)

71.3 ms ± 14.4 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/multiprocessing/pool.py", line 109, in worker
    initializer(*initargs)
  File "/work/dvalsecc/Clustering/DeepSuperCluster/Training/global_model/awk_data.py", line 287, in process
    output_q.put(out)
  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/multiprocessing/queues.py", line 367, in put
    with self._wlock:
  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterru

In [15]:
cfg = config["dataset_conf"]["training"]
cfg["maxevents"] = 512000
cfg["chunk_size"] = 512*10
cfg["batch_size"] = 512
cfg["nworkers"] = 6
cfg["max_batches_in_memory"] = 300

train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**cfg))
it = iter(train_ds)
                                
result = %timeit -n 100 -r 10 -o next(it)

56.2 ms ± 14.4 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


Process ForkPoolWorker-31:
Process ForkPoolWorker-29:
Process ForkPoolWorker-30:
Process ForkPoolWorker-27:
Process ForkPoolWorker-28:
Process ForkPoolWorker-32:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/multip

In [6]:
cfg = config["dataset_conf"]["training"]
cfg["maxevents"] = 512000
cfg["batch_size"] = 512
cfg["nworkers"] = 2
cfg["max_batches_in_memory"] = 50

train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**cfg))
it = iter(train_ds)
                                
result = %timeit -n 100 -r 10 -o next(it)
                                

91 ms ± 14.7 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [7]:
cfg = config["dataset_conf"]["training"]
cfg["maxevents"] = 512000
cfg["batch_size"] = 512
cfg["nworkers"] = 2
cfg["max_batches_in_memory"] = 100

train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**cfg))
it = iter(train_ds)
                                
result = %timeit -n 100 -r 10 -o next(it)

99.1 ms ± 16.1 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [14]:
result

In [6]:
126 / (1000000 / 521)

0.065646

In [4]:
%%time

for x,y, w in ds_train:
    pass

2022-09-25 12:28:04.797692: W tensorflow/core/framework/op_kernel.cc:1768] UNKNOWN: AttributeError: 'LoaderConfig' object has no attribute 'reweight_file'
Traceback (most recent call last):

  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/work/dvalsecc/Clustering/DeepSuperCluster/Training/global_model/awk_data.py", line 564, in _gen
    file_loader_generator = load_batches_from_files_generator(config, preprocessing)

  File "/work/dvalsecc/Clustering/DeepSuperClust

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_12_device_/job:localhost/replica:0/task:0/device:CPU:0}} AttributeError: 'LoaderConfig' object has no attribute 'reweight_file'
Traceback (most recent call last):

  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/work/dvalsecc/miniconda3/envs/clustering_tf2.3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/work/dvalsecc/Clustering/DeepSuperCluster/Training/global_model/awk_data.py", line 564, in _gen
    file_loader_generator = load_batches_from_files_generator(config, preprocessing)

  Fil

# Checks on Nans and inf

In [11]:
for i,(X,Y,w) in enumerate(ds_train()):
    mask = X[4] -1
    mask[mask==-1] = 1
    mask_rec = X[5]-1
    mask_rec[mask_rec==-1] = 1
    s = np.sum(X[0] * mask[:,:,None]) + np.sum(Y[2] * mask[:,:,None]) + np.sum(X[2]*mask_rec[:,:,:,None])
    if s != 0:
        break
    else:
        print(".", end="")

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [4]:
for j, (x,y ,w) in enumerate(ds_train):
    print(j, end="> ")
    found = False
    for i in [0,1,2,4,5]:
        a = tf.reduce_sum(tf.cast(tf.math.is_nan(x[i]), tf.int64)) + tf.reduce_sum(tf.cast(tf.math.is_inf(x[i]), tf.int64)) 
        print(a, end=", ")
        if a.numpy() != 0:
            found = True
    print()
    if found : break
    

0> tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), 
1> tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), 
2> tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), 
3> tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), 
4> tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), 
5> tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, sh

In [5]:
############### 
# Loading the model file
 # Load model modules
spec = importlib.util.spec_from_file_location("model", model_file)
model_lib = importlib.util.module_from_spec(spec)
spec.loader.exec_module(model_lib)

tf.keras.backend.clear_session()
# Construction of the model in the strategy scope
with strategy.scope():
    print(">>> Creating the model")
    # Build the model with all the configs
    model = model_lib.DeepClusterGN(**config)

    #optimizer
    if config['opt'] == "adam":
        opt = tf.keras.optimizers.Adam(learning_rate=config['lr'])

    #compile the model
    model.compile(optimizer=opt)
    model.set_metrics()

    for X, y ,w  in ds_train:
        # Load the model
        ypred = model(X, training=False)
        #l = custom_loss(y, ypred)
        break

    model.summary()
    

>>> Creating the model
Model: "deep_cluster_gn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 graph_builder (GraphBuildin  multiple                 78267     
 g)                                                              
                                                                 
 GHN_global (GHConvI)        multiple                  8256      
                                                                 
 SA_clclass (SelfAttentionBl  multiple                 24960     
 ock)                                                            
                                                                 
 dense_clclass (Sequential)  (256, 24, 1)              90753     
                                                                 
 SA_windclass (SelfAttention  multiple                 41344     
 Block)                                                          
                            

In [ ]:
history = model.fit(ds_train,
        validation_data=ds_test, 
        epochs=config['nepochs'],
        steps_per_epoch=120, 
        validation_steps=2,
        verbose=2,
    )

In [7]:
for j, (x,y ,w) in enumerate(ds_train):
    print(j, end="> ")
    found = False
    for i in [0,1,2,4,5]:
        a = tf.reduce_sum(tf.cast(tf.math.is_nan(x[i]), tf.int64)) + tf.reduce_sum(tf.cast(tf.math.is_inf(x[i]), tf.int64))
        print(a, end=", ")
        if a.numpy() != 0:
            found = True
    ypred = model(x)
    for i in [0,1,2]:
        a = tf.reduce_sum(tf.cast(tf.math.is_nan(ypred[0][i]), tf.int64)) + tf.reduce_sum(tf.cast(tf.math.is_inf(ypred[0][i]), tf.int64))
        print(a, end=", ")
        if a.numpy() != 0:
            found = True
    print()
    if found : break
    

0> tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), 
1> tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), 
2> tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), 
3> tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), dtype=int64), tf.Tensor(0, shape=(), 

KeyboardInterrupt: 

In [ ]:
train_ds = awk_data.load_dataset(awk_data.LoaderConfig(**config["dataset_conf"]["training"]))
test_ds = awk_data.load_dataset(awk_data.LoaderConfig(**config["dataset_conf"]["validation"]))
# Create training and validation
ds_train = train_ds.prefetch(tf.data.AUTOTUNE)